# 1) Importing data

In [21]:
from cgitb import text

import pandas as pd
import numpy as np

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [22]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = '../subtask4b_collection_data.pkl' #MODIFY PATH

In [23]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [24]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [25]:
df_collection

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055874,w094r17f,Medline; PMC,COVID-19 increased the risk of ICU-acquired bl...,10.1007/s00134-021-06346-w,PMC7839935,33506379,no-cc,PURPOSE: The primary objective of this study w...,2021-01-27,"Buetti, Niccolò; Ruckly, Stéphane; de Montmoll...",Intensive Care Med,NaN,NaN,NaN,w094r17f,2021-01-27,1611705600
1056107,wtqwghon,MedRxiv; WHO,Relative disease burdens of COVID-19 and seaso...,10.1101/2020.04.22.20073551,NaN,NaN,medrxiv,Comparisons between the mortality burdens of C...,2020-04-27,"Faust, Jeremy; del Rio, Carlos",NaN,NaN,NaN,NaN,wtqwghon,2020-04-27,1587945600
1056228,r5rctvul,Medline; PMC,COVID‐19 and the Gender Gap in Work Hours,10.1111/gwao.12506,PMC7361447,32837019,no-cc,School and daycare closures due to the COVID‐1...,2020-07-02,"Collins, Caitlyn; Landivar, Liana Christin; Ru...",Gend Work Organ,NaN,NaN,NaN,r5rctvul,2020-07-02,1593648000
1056344,yjqt74q0,Medline; PMC,"Prior infection by seasonal coronaviruses, as ...",10.2807/1560-7917.es.2021.26.13.2001782,PMC8017906,33797390,cc-by,BACKGROUND: Children have a low rate of COVID-...,2021-04-01,"Sermet-Gaudelus, Isabelle; Temmam, Sarah; Huon...",Euro Surveill,NaN,NaN,NaN,yjqt74q0,2021-04-01,1617235200


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [26]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = '../subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = '../subtask4b_query_tweets_dev.tsv' #MODIFY PATH

In [27]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [55]:
df_query_train

,post_id,tweet_text,cord_uid,weighted_bm25_topk,in_topx
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,"[htlvpvz5, h7hj64q5, tpovr13h, rwgqkow3, 32z7b...",1.000000
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,"[apqzyln2, 33znyrn8, lvvw1npq, 296ilxyg, fm8ko...",0.000000
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"[jtwb17u8, a0q61mpi, veeavho5, jbpmbm9m, 8hkxb...",1.000000
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,"[lsgm7y5t, l5ogbl5p, 0w9k8iy1, l4y7v729, 9g0lo...",0.333333
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,"[tiqksd69, b0dzhsrh, k7smwz6w, aqbhxv1f, uppez...",1.000000
...,...,...,...,...,...
12848,14248,"""evidence on covid-19 reveals a growing body o...",9169o29b,"[9169o29b, 1y1ik2u9, dgng39yd, l9lni5d3, jgq96...",1.000000
12849,14249,Outdoor lighting has detrimental impacts on lo...,s2bpha8l,"[s2bpha8l, 8a3fp7ym, lvw0cnwj, zueprmdz, nwb7q...",1.000000
12850,14250,"26/ and influenza virus (and other pathogens, ...",atloc9th,"[y7s6qt6j, asj7352n, w19729nl, futzwn98, lavcs...",0.000000
12851,14251,does it?'sars-cov-2-naïve vaccinees had a 13.0...,t4y1ylb3,"[t4y1ylb3, 7a543f7v, a4klrp3h, o86wki37, sjsaw...",1.000000


In [29]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [30]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [31]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# Tokenization

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import re
from rank_bm25 import BM25Okapi

# Download necessary NLTK resources (if not already downloaded)
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Michael_PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Michael_PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Michael_PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [33]:
def standard_tokenizer(text):
    """Standard tokenizer with stemming and stop word removal."""
    if not isinstance(text, str):
        return []
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text.lower())
    stop_words = set(nltk.corpus.stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    stemmer = SnowballStemmer('english')
    return [stemmer.stem(word) for word in filtered_tokens]

def name_tokenizer(text):
    """Tokenizer for extracting potential names (capitalized n-grams)."""
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    name_ngrams = []
    for n in [2, 3]:  # Bigrams and trigrams
        for i in range(len(pos_tags) - n + 1):
            ngram_pos = pos_tags[i:i + n]
            ngram_words = [word for word, tag in ngram_pos]
            ngram_tags = [tag for word, tag in ngram_pos]
            # Check if the current n-gram potentially represents a name or organization
            # Condition 1: All words in the n-gram are proper nouns (NNP)
            #   - This is common for multi-word names like "Albert Einstein" or "World Health Organization"
            # Condition 2: (For bigrams only) The first word is a proper noun (NNP)
            #   and the second word is either a common noun (NN) or another proper noun (NNP)
            #   - This can capture patterns like "Dr. Smith" or "Apple Inc."
            if all(tag == 'NNP' for tag in ngram_tags) or \
               (n == 2 and ngram_tags[0] == 'NNP' and ngram_tags[1] in ['NN', 'NNP']):
                    name_ngrams.append(" ".join(ngram_words).lower())
    if not name_ngrams:
        return []
    return name_ngrams

def author_metadata_tokenizer(text):
    """Tokenizer for extracting authors from specific given format from paper metadata and creating bi-grams"""
    fullname_tokens = text.split("; ")
    unigram_tokens = [token.split(", ") for token in fullname_tokens]
    unigram_tokens = [token for pair in unigram_tokens for token in pair] # Flatten list
    for token in unigram_tokens:
        token = token.split(' ')[0] # Remove Surname addons e.g. Daniel R.
        token = token.replace('?', '')
        token = token.lower()
    bigram_tokens = [token.replace(',', '') for token in fullname_tokens]
    bigram_tokens = [token.lower() for token in bigram_tokens]
    return unigram_tokens + bigram_tokens
    
def journal_metadata_tokenizer(text):
    """Tokenizer for creating ngrams from journal names"""
    tokens = standard_tokenizer(text)
    multigrams = []
    for n in range(0, len(tokens) + 1):
        for i in range(len(tokens) - n + 1):
            ngram = tokens[i: i + n]
            ngram = ' '.join(ngram)

            # Skip single chars
            if len(ngram) <= 1:
                continue

            multigrams.append(ngram)
    return multigrams

def extract_year(text):
    """Extracts a year (4 digits) from text."""
    match = re.search(r'\b\d{4}\b', text)
    if match:
        return int(match.group(0))
    return []

In [39]:
# Initialize BM25 models for different metadata attributes

df_collection_cleaned = df_collection.fillna('')

tokenized_titles_series = df_collection_cleaned['title'].apply(standard_tokenizer)
tokenized_abstracts_series = df_collection_cleaned['abstract'].apply(standard_tokenizer)
tokenized_authors_series = df_collection_cleaned['authors'].apply(author_metadata_tokenizer)
tokenized_journals_series = df_collection_cleaned['journal'].apply(journal_metadata_tokenizer)
# tokenized_time_series = df_collection_cleaned['time'].apply(extract_year)

bm25_title = BM25Okapi(tokenized_titles_series)
bm25_abstract = BM25Okapi(tokenized_abstracts_series)
bm25_authors = BM25Okapi(tokenized_authors_series)
bm25_journal = BM25Okapi(tokenized_journals_series)
# bm25_time = BM25Okapi(tokenized_time_series)


merged_tokenized_corpus = tokenized_titles_series + tokenized_abstracts_series + tokenized_authors_series + tokenized_journals_series
bm25 = BM25Okapi(merged_tokenized_corpus)

cord_uids = df_collection['cord_uid'].tolist()

# Evaluating merged metadata BM25

In [17]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:5]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [18]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))

In [19]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [20]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

Results on the train set: {1: np.float64(0.15895121761456468), 5: np.float64(0.21320573666329518), 10: np.float64(0.21320573666329518)}
Results on the dev set: {1: np.float64(0.16428571428571428), 5: np.float64(0.21404761904761901), 10: np.float64(0.21404761904761901)}


# Individual metadata BM25 scoring 

In [40]:
def get_individual_scores(query_text):
    """
    Calculates individual BM25 scores for a query against all papers.
    """

    tokenized_query_standard = standard_tokenizer(query_text)
    tokenized_query_names = name_tokenizer(query_text)

    # BM25 scores for different metadata
    bm25_scores_title = bm25_title.get_scores(tokenized_query_standard)
    bm25_scores_abstract = bm25_abstract.get_scores(tokenized_query_standard)
    bm25_scores_authors = bm25_authors.get_scores(tokenized_query_standard + tokenized_query_names)
    bm25_scores_journal = bm25_journal.get_scores(tokenized_query_standard + tokenized_query_names)

    # Return individual scores
    return {
        'title_bm25': bm25_scores_title,
        'abstract_bm25': bm25_scores_abstract,
        'authors_bm25': bm25_scores_authors,
        'journal_bm25': bm25_scores_journal
    }

In [41]:
# Calculate and store individual scores for each tweet in the dev set
df_query_dev['individual_scores'] = df_query_dev['tweet_text'].apply(get_individual_scores)

In [42]:
df_query_dev['individual_scores']

0       {'title_bm25': [1.7974183621509057, 0.0, 0.0, ...
1       {'title_bm25': [13.587086449502365, 0.0, 0.0, ...
2       {'title_bm25': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       {'title_bm25': [1.7974183621509057, 0.0, 0.0, ...
4       {'title_bm25': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
1395    {'title_bm25': [3.767834226949666, 0.0, 0.0, 0...
1396    {'title_bm25': [4.393546311314532, 0.0, 0.0, 0...
1397    {'title_bm25': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1398    {'title_bm25': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1399    {'title_bm25': [3.767834226949666, 0.0, 0.0, 0...
Name: individual_scores, Length: 1400, dtype: object

In [44]:
def rank_papers_with_weights(individual_scores, weights, k=5):
    """
    Calculates weighted similarity using individual scores
    and returns top-k cord_uids.
    """
    num_documents = len(df_collection) # Assumes df_collection is available
    final_similarity = np.zeros(num_documents)

    for key, score_array in individual_scores.items():
        if len(score_array) < num_documents:
             padded_score_array = np.zeros(num_documents)
             padded_score_array[:len(score_array)] = score_array
             current_score_array = padded_score_array
        else:
             current_score_array = score_array

        if key in weights:
             current_score_array = np.nan_to_num(current_score_array, nan=0.0, posinf=1e10, neginf=-1e10)
             final_similarity += weights[key] * current_score_array
        else:
             print(f"Warning: No weight found for key '{key}'. Field ignored.")


    final_similarity = np.nan_to_num(final_similarity, nan=0.0, posinf=1e10, neginf=-1e10)

    top_indices = np.argsort(-final_similarity)[:k]

    return [cord_uids[i] for i in top_indices]

In [45]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(
            lambda x: (1 / ([i for i in x[col_pred][:k]].index(x[col_gold]) + 1)
                      if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [47]:
# Define the range or specific values to explore for each weight
weight_options = {
    'title_bm25': [0.3, 0.5, 1.0],
    'abstract_bm25': [0.3, 0.5, 1.0],
    'authors_bm25': [0.3, 0.5, 1.0],
    'journal_bm25': [0.3, 0.5, 1.0],
    'publish_time_sim': [0.3, 0.5, 1.0],
}

In [48]:
import itertools

best_mrr = -1
best_weights = None

# Generate all combinations of weights
weight_combinations = list(itertools.product(*weight_options.values()))

print(f"Exploring {len(weight_combinations)} weight combinations with Grid Search...")

for combo in weight_combinations:
    current_weights = dict(zip(weight_options.keys(), combo))

    # Apply the ranking function with current weights
    df_query_dev['preds_grid_search'] = df_query_dev['individual_scores'].apply(
        lambda x: rank_papers_with_weights(x, current_weights, k=5)
    )

    # Evaluate the results (with MRR@5 as the objective)
    mrr_at_5 = get_performance_mrr(df_query_dev, 'cord_uid', 'preds_grid_search', list_k=[5])[5]

    if mrr_at_5 > best_mrr:
        best_mrr = mrr_at_5
        best_weights = current_weights
        print(f"New best weights: {current_weights}, MRR@5: {mrr_at_5}")

print("\nGrid Search Finished.")
print(f"Best MRR@5: {best_mrr}")
print(f"Best Weights: {best_weights}")

Exploring 243 weight combinations with Grid Search...
New best weights: {'title_bm25': 0.3, 'abstract_bm25': 0.3, 'authors_bm25': 0.3, 'journal_bm25': 0.3, 'publish_time_sim': 0.3}, MRR@5: 0.6110357142857143
New best weights: {'title_bm25': 0.3, 'abstract_bm25': 0.5, 'authors_bm25': 0.3, 'journal_bm25': 0.3, 'publish_time_sim': 0.3}, MRR@5: 0.6245833333333333

Grid Search Finished.
Best MRR@5: 0.6245833333333333
Best Weights: {'title_bm25': 0.3, 'abstract_bm25': 0.5, 'authors_bm25': 0.3, 'journal_bm25': 0.3, 'publish_time_sim': 0.3}


## Training set evaluation

In [49]:
def get_weighted_similarity_topk(query_text,
                        weight_title=0.3,
                        weight_abstract=0.5,
                        weight_authors=0.3,
                        weight_journal=0.3):
    """
    Calculates a weighted similarity score using BM25 on different metadata.
    """

    # BM25 scores for different metadata
    tokenized_query_standard = standard_tokenizer(query_text)
    tokenized_query_names = name_tokenizer(query_text)

    bm25_scores_title = bm25_title.get_scores(tokenized_query_standard)
    bm25_scores_abstract = bm25_abstract.get_scores(tokenized_query_standard)
    bm25_scores_authors = bm25_authors.get_scores(tokenized_query_names)
    bm25_scores_journal = bm25_journal.get_scores(tokenized_query_names)

    # Combine weighted BM25 scores and publish time similarity
    final_bm25_score = (
        weight_title * bm25_scores_title +
        weight_abstract * bm25_scores_abstract +
        weight_authors * bm25_scores_authors +
        weight_journal * bm25_scores_journal
    )

    top_indices = np.argsort(-final_bm25_score)[:5]
    return df_collection['cord_uid'].iloc[top_indices].tolist()

In [50]:
# Retrieve topk candidates using BM25
df_query_train['weighted_bm25_topk'] = df_query_train['tweet_text'].apply(get_weighted_similarity_topk)

# 3) Evaluating BM25 with preprocessing

In [51]:
# Evaluate BM25 results with best weights
best_weights = {'title_bm25': 1.0, 'abstract_bm25': 0.7, 'authors_bm25': 0.3, 'journal_bm25': 0.3, 'publish_time_sim': 0.1}

df_query_dev['weighted_bm25_topk'] = df_query_dev['individual_scores'].apply(
    lambda x: rank_papers_with_weights(x, best_weights, k=5)
)

results_train_weighted_bm25 = get_performance_mrr(df_query_train, 'cord_uid', 'weighted_bm25_topk')
results_dev_weighted_bm25 = get_performance_mrr(df_query_dev, 'cord_uid', 'weighted_bm25_topk')

In [52]:
# Print BM25 MRR@k results
print(f"BM25 Results on the train set: {results_train_weighted_bm25}")
print(f"BM25 Results on the dev set: {results_dev_weighted_bm25}")

BM25 Results on the train set: {1: np.float64(0.5713841126585233), 5: np.float64(0.6234536684042636), 10: np.float64(0.6234536684042636)}
BM25 Results on the dev set: {1: np.float64(0.5514285714285714), 5: np.float64(0.6051785714285715), 10: np.float64(0.6051785714285715)}


# 4) Exporting results to prepare the submission on Codalab

In [53]:
df_query_dev['preds'] = df_query_dev['weighted_bm25_topk'].apply(lambda x: x[:5])

In [54]:
df_query_dev[['post_id', 'preds']].to_csv('predictions_weighted_bm25.tsv', index=None, sep='\t')